In [29]:
import numpy as np

def recuit_simulé(f, x0,*args, q=0.2, c0=1e-5, alpha=0.99, max_iter=2000):
    # q = variance de la gaussienne
    # c0 = critère d'arret
    # alpha = taux d'acceptation

    for _ in range(max_iter):
        x_new = np.random.normal(x0, q)  # Tirez un nouveau x à partir d'une distribution q(x|xk)

        delta_f = f(x_new,args) - f(x0,args)

        """
        La probabilité que xk +1 soit accepé est :
                    1 si f (xk +1 ) ≤f (xk ) ;
                    exp(−f (xk +1 )−f (xk ) / ck) sinon;
        """
        if delta_f <= 0 or np.random.rand() < np.exp(-delta_f / c0):
            x0 = x_new

        """
        Parametre de controle initialement grand 
        (grand taux d’acceptation) puis
        décroissance lente : 
        ck +1 = α * ck 
        avec α = 0.99
        """
        c0 *= alpha 

    result = {'x': x0, 'fun': f(x0,args)}
    return result

In [36]:
def example_function(x):
    return x[0]**2+x[1]**2+x[2]**3

In [28]:
x_optimal = recuit_simulé(example_function, x0=[10,10])

print(f'x : {x_optimal["x"]}')
print(f'fun : {x_optimal["fun"]}')


x : [0.00080254 0.0016688 ]
fun : 3.4289567906542693e-06


In [30]:
import numpy as np

class Particle:
    def __init__(self, dim, min_bound, max_bound):
        self.position = np.random.uniform(min_bound, max_bound, dim)
        self.velocity = np.random.uniform(-1, 1, dim)
        self.best_position = np.copy(self.position)
        self.best_score = float('inf')

def particle_swarm_optimization(f, dim, min_bound, max_bound, swarm_size, max_iter, w, c1, c2):
    swarm = [Particle(dim, min_bound, max_bound) for _ in range(swarm_size)]
    global_best_position = None
    global_best_score = float('inf')

    for _ in range(max_iter):
        for particle in swarm:
            score = f(particle.position)

            if score < particle.best_score:
                particle.best_score = score
                particle.best_position = np.copy(particle.position)

            if score < global_best_score:
                global_best_score = score
                global_best_position = np.copy(particle.position)

        for particle in swarm:
            r1, r2 = np.random.rand(dim), np.random.rand(dim)
            inertia = w * particle.velocity
            cognitive = c1 * r1 * (particle.best_position - particle.position)
            social = c2 * r2 * (global_best_position - particle.position)
            particle.velocity = inertia + cognitive + social
            particle.position += particle.velocity

            # Assurez-vous que les particules restent dans les limites définies
            particle.position = np.clip(particle.position, min_bound, max_bound)

    return global_best_position



In [39]:
optimal_solution = particle_swarm_optimization(
    f=example_function,
    dim=3,
    min_bound=-5.0,
    max_bound=5.0,
    swarm_size=30,
    max_iter=2000,
    w=0.5,
    c1=10,
    c2=10,
)

print("Solution optimale:", optimal_solution)
print("Valeur de la fonction objectif à la solution optimale:", example_function(optimal_solution))


Solution optimale: [ 0.03524217  0.19471326 -5.        ]
Valeur de la fonction objectif à la solution optimale: -124.96084473599554
